In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

import importlib

import utilis

Using TensorFlow backend.


In [2]:
# load numpy array from csv file
from numpy import loadtxt
# load array
X_train = loadtxt('x_train2.csv', delimiter=',')
Y_train = loadtxt('y_train2.csv', delimiter=',')
# print the array
X_train
print(X_train.shape)

(2000, 1000)


In [3]:
Y_train
print(Y_train.shape)

(2000, 41)


In [4]:
VOCAB_SIZE = 1254
INPUT_LENGTH = 1000
EMBEDDING_DIM = 128

In [8]:
# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(CuDNNLSTM(128)))
#     model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(41, activation='softmax'))
    return model

In [9]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 41)                10537     
Total params: 435,241
Trainable params: 435,241
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# optimize for GPU
from keras.utils import multi_gpu_model

model_GPU = multi_gpu_model(model, gpus= 4)
model_GPU.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# training
epochs = 50
batch_size = 64

history = model_GPU.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 1800 samples, validate on 200 samples
Epoch 1/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.5478 - acc: 0.4817 - val_loss: 2.4490 - val_acc: 0.2700
Epoch 2/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.6591 - acc: 0.4467 - val_loss: 2.4134 - val_acc: 0.2500
Epoch 3/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.5682 - acc: 0.4856 - val_loss: 2.4116 - val_acc: 0.3000
Epoch 4/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.5081 - acc: 0.5006 - val_loss: 2.4492 - val_acc: 0.2150
Epoch 5/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.5105 - acc: 0.4983 - val_loss: 2.3964 - val_acc: 0.2450
Epoch 6/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.4597 - acc: 0.5228 - val_loss: 2.4631 - val_acc: 0.2950
Epoch 7/50
1800/1800 [==============================] - 3s 2ms/step - loss: 1.4255 - acc: 0.5294 - val_loss: 2.5326 - val_acc: 0.2600
Epoch 8/50
1800

In [21]:
# save model
model_json = model_GPU.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
# # save loss
LOSS_FILE_NAME = "BiLSTM_goals_small_manu_ep1"
utilis.history_saver_bad(history, LOSS_FILE_NAME)

In [20]:
example_x = X_train[0]
print(np.shape(example_x))
temp = model.predict(X_train[0:10])
# print(len(temp)), temp
print(temp[0])
for i in temp:
    print(np.argmax(i))

(1000,)
[1.04413629e-02 2.80173816e-04 1.03286910e-03 8.76937527e-03
 1.71356180e-04 1.13787306e-02 6.03483795e-06 4.41065367e-06
 6.55875877e-02 9.34809941e-06 9.92909889e-04 6.38116398e-06
 8.55467655e-03 2.76544561e-05 1.06773114e-05 5.91152627e-03
 6.04268163e-02 1.96593348e-04 1.34555385e-05 1.73397581e-04
 9.06704099e-06 6.99635666e-06 3.20220692e-03 9.13570184e-05
 3.40959802e-02 1.38694914e-02 1.66026934e-03 3.22664477e-04
 9.15775448e-03 7.27174920e-05 2.21225309e-05 3.74296476e-04
 7.91369565e-03 1.16799446e-03 1.63717225e-01 3.51800772e-05
 1.60687734e-04 5.83686590e-01 5.20045171e-03 1.12384884e-03
 1.14099064e-04]
37
37
37
3
32
37
24
5
5
3
